# Nitrogen Concentration and Nitrogen Deposition - part 2

## 4 Extending the initial approach

In the previous sections, we slowly built a model that helped us to simulate the dry deposition of NH$_3$. For reasons of simplicity, we had to make some assumptions. The benefit of those is that now you have a working model, you understand what it does and you know its limitations.

The next step is to take a closer look at some of the assumptions we made. The most relevant assumptions were:

* Deposition on a wet canopy does not experience canopy conductance.
* A constant $LAI$ in the Jarvis approach.
* Stress functions $F_2, F_3$ and $F_4$ in the Jarvis approach all equal 1.
* Parameter values may be uncertain.

When implementing parameterizations to improve the model, it is of course interesting to see how important the changes are. Therefore, always write down a few important statistics (e.g., annual total deposition flux) or keep a few graphs in a separate document. Keep it practical, do not try to keep track of too many things, a few key numbers or figures is usually enough. Let's now improve our model.

### 4.1 Wet canopy

When the vegetation is wet, for example after precipitation or dew, ammonia will deposit on the wet surfaces outside the stomates. As such, the deposition flux will bypass the canopy resistance. This may be simulated with $r_c = 0$ s m$^{-1}$ when the canopy is wet. The canopy wetness ultimately depends on the balance between precipitation, evaporation and dripping. 

### Exercise 4.1: Wet canopy analysis

In the next box we will calculate the deposition rate. Let's keep it simple and assume that the vegetation is wet for a few hours (e.g., 3 h) after the precipitation. In the following cell-boxes, we will make the calculation by taking into account the wet canopy.

In [ ]:
# Load necessary python packages.
import sys
!{sys.executable} -m pip install cufflinks > /dev/null; # Remove > /dev/null in case of errors.

from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cufflinks as cf
import plotly.express as px

In [ ]:
# Load the velocity (u; m/s), global radiation (Rg; W/m^2), rain (Rain; mm), rain in the last three hours 
# (Rain_last3h; mm), leaf area index (LAI; m^2/m^2), aerodynamical, canopy and total resistance 
# (ra, rc, rt; s/m) and dry nitrogen deposition for three stations (F_NH3_W, F_NH3_V, F_NH3_Z; ug/m^2/s).
df_meteo1 = pd.read_csv('testiiiiiiiiing.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'u', 'Rg', 'Rain', 'Rain_last3h', 'LAI', 'ra', 'rc', 'rt', 
    'F_NH3_W', 'F_NH3_V', 'F_NH3_Z'], parse_dates=['date-time'])
df_meteo1.columns = ['u', 'Rg', 'Rain', 'Rain_last3h', 'LAI', 'ra', 'rc', 'rt', 'F_NH3_W', 'F_NH3_V', 'F_NH3_Z']

# Load the values of nitrogen NH3 concentration for three different stations (Wekerom, Vredepeel and Zegveld). 
# Data are loaded as hourly values in ug/m3.
df_Wekerom = pd.read_csv('BasicSkills_Wekerom.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Wekerom.columns = ['NH3_W']

df_Vredepeel = pd.read_csv('BasicSkills_Vredepeel.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Vredepeel.columns = ['NH3_V']

df_Zegveld = pd.read_csv('BasicSkills_Zegveld.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Zegveld.columns = ['NH3_Z']

# Save the data from Wekerom, Vredepeel and Zegveld in the same data frame.
df_result = pd.concat([df_Wekerom, df_Vredepeel, df_Zegveld], axis=1, sort=False)

Now we will calculate the total resistance having in mind rain in the last 3 hours and the deposition rate for the three stations (i.e., for Wekerom, Vredepeel and Zegveld).

In [ ]:
# Make a new data frame that includes only the rain in the last 3 hours (Rain_last3h; mm) and 
# the canopy resistance we calculated in the section 3 (rc; s/m).
wet_canopy = df_meteo1[['Rain_last3h', 'rc']]

# Find the position where the rain in the last 3 hours is more than 0 mm. 
# If there is rain in the last 3 hours then set the canopy resistance at those time steps to 0 s/m.
wet_canopy['rc'][wet_canopy['Rain_last3h']>0.] = 0.

# Set up the sub-laminar boundary layer resistance (rb; s/m).
rb = 5.

# Calculate new total resistance (rt; s/m) when including rain over the last 3 hours.
rt_wet_canopy = df_meteo1['ra'] + rb + wet_canopy['rc']

# Add the calculated (new) total resistance into a matrix with the meteorological data.
df_meteo1 = pd.concat([df_meteo1, rt_wet_canopy], axis=1, sort=False)
df_meteo1.columns = ['u', 'Rg', 'Rain', 'Rain_last3h', 'LAI', 'ra', 'rc', 'rt', 
                    'F_NH3_W', 'F_NH3_V', 'F_NH3_Z', 'rt_wet_canopy']

# Calculate the deposition rate using a gradient resistance model and a new total resistance 
# for all three stations, i.e., for Wekerom, Vredepeel and Zegveld (F_wet_canopy_NH3_W, F_wet_canopy_NH3_V
# and F_wet_canopy_NH3_Z, respectively; ug/m^2/s).
F_wet_canopy_NH3_W = df_result['NH3_W']/rt_wet_canopy
F_wet_canopy_NH3_W.columns = ['F_wet_canopy_NH3_W']

F_wet_canopy_NH3_V = df_result['NH3_V']/rt_wet_canopy
F_wet_canopy_NH3_V.columns = ['F_wet_canopy_NH3_V']

F_wet_canopy_NH3_Z = df_result['NH3_Z']/rt_wet_canopy
F_wet_canopy_NH3_Z.columns = ['F_wet_canopy_NH3_Z']

# Add the calculated (new) deposition rate into a matrix with the meteorological data.
df_meteo1 = pd.concat([df_meteo1, F_wet_canopy_NH3_W, F_wet_canopy_NH3_V, F_wet_canopy_NH3_Z], axis=1, sort=False)
df_meteo1.columns = ['u', 'Rg', 'Rain', 'Rain_last3h', 'LAI', 'ra', 'rc', 'rt', 'F_NH3_W', 'F_NH3_V', 'F_NH3_Z',
                   'rt_wet_canopy', 'F_wet_canopy_NH3_W', 'F_wet_canopy_NH3_V', 'F_wet_canopy_NH3_Z']

In [ ]:
# Plot the deposition rate when including rain over the last 3 hours in the calculation of 
# the total resistance. Make a figure for the three stations
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [years] on x-axis and Deposition rate [um/m^2/s] on y-axis. 
fig1 = df_meteo1[['F_wet_canopy_NH3_W', 'F_wet_canopy_NH3_V', 'F_wet_canopy_NH3_Z']].iplot(asFigure=True, 
    subplots=True, shape=(3,1), shared_xaxes=True, fill=True, 
    layout=dict(yaxis=dict(title=' '), xaxis=dict(title=' ')))
fig1['layout']['yaxis2'].update({'title':'Deposition rate [ug/m2/s]'})
fig1['layout']['xaxis2'].update({'title':' '})    
fig1['layout']['xaxis3'].update({'title':'Time [years]'})
fig1['layout']['yaxis1'].update({'range': [0, 20]}) 
fig1['layout']['yaxis2'].update({'range': [0, 20]}) 
fig1['layout']['yaxis3'].update({'range': [0, 20]}) 
fig1.show()

### Question 4.1: How does the deposition rate change in the wet canopy between the three stations?

#### Your answer is:

### Question 4.2: How does the total annual deposition change between the three stations? How does this total deposition change in contrast to the total annual deposition when we have a dry canopy?

To answer this question, you need to calculate the total deposition rate over a year for three stations assuming that we have a dry canopy (i.e., when we do not look at the rain over the last three hours) and that we have a wet canopy (i.e., when we look at the rain over the last three hours). We will again set up the python program for you and make the plots. Make your conclusions by analizing the figure and the table below. The result is expressed in g m$^{-2}$yr$^{-1}$.

In [ ]:
# Calculate the total annual deposition for the three stations. 
df_meteo_wet_canopy_deposition_totalannual = 3600./1000000*df_meteo1[['F_NH3_W', 'F_NH3_V', 'F_NH3_Z', 'F_wet_canopy_NH3_W', 'F_wet_canopy_NH3_V', 
    'F_wet_canopy_NH3_Z']].resample('Y').sum()

df_meteo_wet_canopy_deposition_totalannual

In [ ]:
# Plot the result (total annual deposition) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Results for deposition at dry canopy are given with solid lines 
# and results for deposition at wet canopy with dotted lines. 
# Plot Time [years] on x-axis and Deposition [g/m^2/yr] on y-axis. 
fig2 = df_meteo_wet_canopy_deposition_totalannual.iplot(asFigure=True, 
    xTitle="Time [years]", yTitle="Deposition [g/m2/yr]", 
    colors=['orange', 'blue', 'green', 'orange', 'blue', 'green'], 
    dash=['solid', 'solid', 'solid', 'dot', 'dot', 'dot'], width=2)
fig2.show()

#### Your answer to question 4.2 is:

## 4.2 Variable $LAI$

In the calculation for canopy resistance, we started by assuming that $LAI$ is constant. This is not realistic
for most vegetation types, except for evergreen vegetation. A relevant consideration is that in the summer season, the $LAI$ and NH$_3$ concentrations are larger than in the winter period. So there is a correlation that would enlarge the error. To account for variable $LAI$, we will implement a seasonal cycle in the $LAI$, using a cosine function as:

\begin{equation}
LAI_t = A_{LAI}+A_{LAI}\cos\Bigg(\frac{2\pi(t-t_{LAI,max})}{365}\Bigg)
\end{equation}

where $A_{LAI}$ (m$^2$m$^{-2}$)is the amplitude, $t$ (day) refers to the day of year and $t_{LAI,max}$ is the time (a day in a year) when the $LAI$ is the largest. Since the cosine part of the function varies between – $A_{LAI}$ and + $A_{LAI}$, we add one time $A_{LAI}$ to force the function to be positive. The $LAI$ thus varies between 0 and 2$A_{LAI}$. The 365 represents an annual cycle.

### Exercise 4.2: Analysis of Variable LAI

In the next several cells we will first, calculate the periodic change in the $LAI$, second, calculate its influence on the deposition rate, and third, plot the dry deposition rate when the new (more realistic) $LAI$ is taken into account. In the first cell-box from the top you need to write the amplitude of your $LAI$ based on your chosen vegetation type (as you did in exercise 3.2).

In [ ]:
# Copy the meteorological data into a new data frame.
df_meteo_lai = df_meteo1.copy()

# Determine the amplitude of $LAI$ for the cosine function for your chosen vegetation type.
A_lai     = 2.
t_lai_max = 181.

# Calculate the periodic (cosine) change in the $LAI$.
df_meteo_lai['LAI'] = A_lai + A_lai*np.cos(2.*np.pi*(df_meteo_lai.index.dayofyear - t_lai_max) / 365.)

In [ ]:
# Let us perform another calculation of the canopy resistance and finaly of the dry deposition rate.
# Write the values of your chosen parameters.
rcmin = 125. 
Rgl   = 30.
rcmax = 1000.

# Calculate f and the stress function for solar radiation F1.
f_lai     = 0.55*(df_meteo1['Rg']/Rgl)*(2./df_meteo_lai['LAI'])
F1_lai    = ((rcmin/rcmax)+f_lai)/(1+f_lai)

# Calculate the canopy resistance.
rc_lai    = rcmin/(df_meteo_lai['LAI']*F1_lai)

# Calculate new total resistance when including periodic change in LAI.
rt_lai    = df_meteo1['ra'] + rb + rc_lai

# Calculate the deposition rate using a gradient resistance model and new total resistance 
# for all three stations, i.e., for Wekerom, Vredepeel and Zegveld.
F_lai_NH3_W = df_Wekerom['NH3_W']/rt_lai
F_lai_NH3_W.columns = ['F_lai_NH3_W']

F_lai_NH3_V = df_result['NH3_V']/rt_lai
F_lai_NH3_V.columns = ['F_lai_NH3_V']

F_lai_NH3_Z = df_result['NH3_Z']/rt_lai
F_lai_NH3_Z.columns = ['F_lai_NH3_Z']

# Add the calculated canopy resistance and deposition rate into a matrix with the meteorological data.
df_meteo1 = pd.concat([df_meteo1, rc_lai, rt_lai, F_lai_NH3_W, F_lai_NH3_V, F_lai_NH3_Z], axis=1, sort=False)
df_meteo1.columns = ['u', 'Rg', 'Rain', 'Rain_last3h', 'LAI', 'ra', 'rc', 'rt', 'F_NH3_W', 'F_NH3_V', 'F_NH3_Z',
    'rt_wet_canopy', 'F_wet_canopy_NH3_W', 'F_wet_canopy_NH3_V', 'F_wet_canopy_NH3_Z', 'rc_lai', 'rt_lai',
    'F_lai_NH3_W', 'F_lai_NH3_V', 'F_lai_NH3_Z']

In [ ]:
# Plot the deposition rate when including periodic change in LAI in the calculation of 
# the total resistance. Make a figure for the three stations
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [years] on x-axis and Deposition rate [ug/m^2/s] on y-axis. 
fig3 = df_meteo1[['F_lai_NH3_W', 'F_lai_NH3_V', 'F_lai_NH3_Z']].iplot(asFigure=True, 
    subplots=True, shape=(3,1), shared_xaxes=True, fill=True, 
    layout=dict(yaxis=dict(title=' '), xaxis=dict(title=' ')))
fig3['layout']['yaxis2'].update({'title':'Deposition rate [ug/m2/s]'})
fig3['layout']['xaxis2'].update({'title':' '})    
fig3['layout']['xaxis3'].update({'title':'Time [years]'})   
fig3['layout']['yaxis1'].update({'range': [0, 5]}) 
fig3['layout']['yaxis2'].update({'range': [0, 5]}) 
fig3['layout']['yaxis3'].update({'range': [0, 5]}) 
fig3.show()

### Question 4.2.1: Does the vegetation get the NH$_3$ when it needs?

#### Your answer is:

### Question 4.2.2: How much does a variable $LAI$ change the total tearly deposition?

To answer this question, you need to calculate the total deposition rate over a year and  for three stations assuming that the $LAI$ is constant and that it changes as a cosine (periodic) function. Do you think you can make this calculation yourself? Let's try. What you need is look at the code above, select the correct variables and run your cell-box using **Shift+Enter**.

In [ ]:
# Make the calculation in this cell.

#### Your answer is:

### Question 4.2.3: How much does a variable $LAI$ change the seasonal deposition rate?

In [ ]:
# Calculate the seasonal variability deposition rate for the three stations. 
# Firts, select variables you want to calculate the seasonal cycle for from the matrix that contains 
# all meteorological data.
df_meteo_lai_deposition_seasonal = df_meteo1[['F_NH3_W', 'F_NH3_V', 'F_NH3_Z', 'F_lai_NH3_W', 'F_lai_NH3_V', 'F_lai_NH3_Z']]

# Calculate the seasonal cycle.
df_meteo_lai_deposition_seasonal = df_meteo_lai_deposition_seasonal.groupby( 
    2*((df_meteo_lai_deposition_seasonal.index.week-1)//2 + 1)).mean()

In [ ]:
# Plot the result (multi-year seasonal variability) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Results for deposition rate with constant LAI are given with solid lines 
# and results for deposition rate with periodic LAI with dotted lines. 
# Plot Time [weeks] on x-axis and Deposition rate [ug/m^2/s] on y-axis. 
fig4 = df_meteo_lai_deposition_seasonal.iplot(asFigure=True, xTitle="Time [weeks]",
    yTitle="Deposition rate [ug/m2/s]", colors=['orange', 'blue', 'green', 'orange', 'blue', 'green'], 
    dash=['solid', 'solid', 'solid', 'dot', 'dot', 'dot'], width=2)
fig4.show()

#### Your answer to question 4.2.3 is:

## 4.3 Variable stress functions $F_2$, $F_3$ and $F_4$

In exercises 3.2 and 3.4, we worked with a single stress function $F_1$. This stress function describes the sensitivity to solar radiation, and thus implements a diurnal cycle in the calculation for the canopy resistance (something that you should have noticed on the figure). The stress functions $F_2$, $F_3$ and $F_4$ describe the sensitivity of the canopy resistance to vapor pressure deficit, temperature and soil moisture. These stressors may act significantly to increase the canopy resistance in the summer period and particularly during droughts and heat waves.

### Exercise 4.3: Analysis of the Influence of Variable Stress Functions on the Deposition Rate
To see the influence of the three stress functions are calculated in the cell below along with their influence on the canopy resistance and the deposition rate.

In [ ]:
# Load the additional meteorological data. 
# Load the air temperature, humidity, and the soil water content at 65, 125, 250 and 500 mm
df_meteo_add = pd.read_csv('BasicSkills_meteo2.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'Air_temp', 'Humidity', 'Water_cont_65', 
    'Water_cont_125', 'Water_cont_250', 'Water_cont_500'], parse_dates=['date-time'])
df_meteo_add.columns = ['Air_temp', 'Humidity', 'Water_cont_65', 
    'Water_cont_125', 'Water_cont_250', 'Water_cont_500']

rcmin = 125. 
Rgl   = 30.
LAI   = 6.4
rcmax = 1000.

# Calculate f and the stress function for solar radiation F1.
f_stress     = 0.55*(df_meteo1['Rg']/Rgl)*(2./LAI)
F1_stress    = ((rcmin/rcmax)+f_stress)/(1+f_stress)

# Calculate the stress function for vapor pressure deficit F2. 
# First define some constants; gass constant for dry air (Rd; J/kgK), gass constant for water vapor (Rv; J/kgK),
# and pressure (p; Pa), and parameter connected with water vapour deficit (hs; -). 
# Write the value of the parameter hs that corresponds to your chosen vegetation type.
hs   = 47.35
Rd   = 287.
Rv   = 462.
p    = 101300.

# Calculate saturation (qs; g/kg) and actual (qa; g/kg) water vapor mixing ratios.
qs = Rd/Rv*(0.6107*10**(7.5*(df_meteo_add['Air_temp'])/(237.3+df_meteo_add['Air_temp'])))/p

#df_meteo_add.loc[df_meteo_add['Humidity']  > 500,'Humidity']  = np.nan
qa = df_meteo_add['Humidity']*qs

# Calculate the stress function for temperature F2.
F2 = 1./(1.+hs*(qs-qa))

# Calculate the stress function for soil moisture F3. 
# First define the optimal air temperature for photosynthesis (Tref; K).  
Tref = 25.

# Calculate F3. 
F3 = 1.-0.00168*(Tref-df_meteo_add['Air_temp'])**2.

# Calculate F4. First define some parameters. Parameters d1, ..., d4 and dtot are defined for you.
# Look up theta_wilt and theta_ref in the Table 2 of Kumar et al. (2011) paper.
theta_wilt = 0.047
theta_ref  = 0.434
d1         = 0.095
d2         = 0.093
d3         = 0.188
d4         = 0.2
dtot       = 0.575

# Calculate denominator and F4.
denom = (theta_ref - theta_wilt)*dtot

F4 = ((df_meteo_add['Water_cont_65']-theta_wilt)*d1+(df_meteo_add['Water_cont_125']-theta_wilt)*d2+
    (df_meteo_add['Water_cont_250']-theta_wilt)*d3+(df_meteo_add['Water_cont_500']-theta_wilt)*d4)/denom

In [ ]:
# Calculate the canopy resistance.
rc_stress    = df_meteo1['rc']/(F1_stress*F2*F3*F4)

# Calculate new total resistance (rt_stress; s/m) when including three stress functions (F2, F3 and F4).
rt_stress    = df_meteo1['ra'] + rb + rc_stress

# Calculate the deposition rate using a gradient resistance model and new total resistance 
# for all three stations, i.e., for Wekerom, Vredepeel and Zegveld.
F_stress_NH3_W = df_result['NH3_W']/rt_stress
F_stress_NH3_W.columns = ['F_stress_NH3_W']

F_stress_NH3_V = df_result['NH3_V']/rt_stress
F_stress_NH3_V.columns = ['F_stress_NH3_V']

F_stress_NH3_Z = df_result['NH3_Z']/rt_stress
F_stress_NH3_Z.columns = ['F_stress_NH3_Z']

# Add the calculated canopy resistance and deposition rate into a matrix with the meteorological data.
df_meteo1 = pd.concat([df_meteo1, rc_stress, rt_stress, F_stress_NH3_W, F_stress_NH3_V, F_stress_NH3_Z], axis=1, sort=False)
df_meteo1.columns = ['u', 'Rg', 'Rain', 'Rain_last3h', 'LAI', 'ra', 'rc', 'rt', 'F_NH3_W', 'F_NH3_V', 'F_NH3_Z',
    'rt_wet_canopy', 'F_wet_canopy_NH3_W', 'F_wet_canopy_NH3_V', 'F_wet_canopy_NH3_Z', 'rc_lai', 'rt_lai',
    'F_lai_NH3_W', 'F_lai_NH3_V', 'F_lai_NH3_Z', 'rc_stress', 'rt_stress', 
    'F_stress_NH3_W', 'F_stress_NH3_V', 'F_stress_NH3_Z']


In [ ]:
# Plot the deposition rate when including the stress functions in the calculation of 
# the canopy resistance. Make a figure for the three stations
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [years] on x-axis and Deposition rate [um/m^2/s] on y-axis. 
fig5 = df_meteo1[['F_stress_NH3_W', 'F_stress_NH3_V', 'F_stress_NH3_Z']].iplot(asFigure=True, 
    subplots=True, shape=(3,1), shared_xaxes=True, fill=True, 
    layout=dict(yaxis=dict(title=' '), xaxis=dict(title=' ')))
fig5['layout']['yaxis2'].update({'title':'Deposition rate [ug/m2/s]'})
fig5['layout']['xaxis2'].update({'title':' '})    
fig5['layout']['xaxis3'].update({'title':'Time [years]'})  
fig5['layout']['yaxis1'].update({'range': [0, 8]}) 
fig5['layout']['yaxis2'].update({'range': [0, 8]}) 
fig5['layout']['yaxis3'].update({'range': [0, 8]}) 
fig5.show()

### Question 4.3.1: How does the deposition rate change in comparisson to the dry deposition rate analyzed in the section 3.4?

#### Your answer is:

### Exercise 4.4: Variability in NH$_3$ deposition rate

In this exercise, you will analyze the variability in NH$_3$ deposition rate. You will look at the variability on a. inter-annual, b. seasonal and c. daily scales. You will compare the results obtained at the three stations and results between the simple model (that we built in the first part of basic skills) and models we built here. Below each of the figures you will again find some questions you can answer by analyzing your figures.

Note that here we only briefly help you with your model. Most of the calculation will be set up by you. 

### Exercise 4.4a: Inter-annual variability

In this exercise, you will look at the variability in the deposition rate on inter-annual scale. In the cell-box below follow our instructions written in the comments (remember that comments start with the sign #), and finish all python commands.

In [ ]:
# Let's first select the data for analysis. Data will be written in a new dataframe. 
# This command is written for you. 
# You can use '#' sign to exclude the data you do not want to analyse. That way you can slowly build up your analysis
# from the data including only F1 (first row), data including wet canopy (second row), data including 
# variable LAI (thirs row) and data including all stress functions (last row).
df_meteo1_interannual = df_meteo1[[
    'F_NH3_W', 'F_NH3_V', 'F_NH3_Z',
#    'F_wet_canopy_NH3_W', 'F_wet_canopy_NH3_V', 'F_wet_canopy_NH3_Z', 
#    'F_lai_NH3_W', 'F_lai_NH3_V', 'F_lai_NH3_Z', 
    'F_stress_NH3_W', 'F_stress_NH3_V', 'F_stress_NH3_Z']]

# Calculate the inter-annual variability for the three stations. 
# Inter-annual variability is calculated as a mean value over each year of our time-series.
# To make a calculation have a look at the Exercise 2.2a from Basic Skills, part 1.
df_meteo1_mean_interannual = 

# Let's make a plot.
# Plot the result (inter-annual variability) for the three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [years] on x-axis and Deposition rate [um/m^2/s] on y-axis.  
fig6 = df_meteo1_mean_interannual.iplot(asFigure=True, xTitle="Time [years]", yTitle="Deposition rate [um/m3/s]", 
    colors=['orange', 'blue', 'green', 'orange', 'blue', 'green'], 
    dash=['solid', 'solid', 'solid', 'dot', 'dot', 'dot'], width=2)
fig6.update_xaxes(tickvals=[2014, 2015, 2016, 2017, 2018, 2019])
fig6.show()

### Question 4.4a: How does the deposition vary on inter-annual scale when we transition from a simple to a realistic canopy resistance calculation?

#### Your answer is:

### Exercise 4.4b: Multi-year seasonal variability

In this exercise, you will look at the variability in the deposition rate on seasonal scale. In the cell-box below again follow our instructions written in the comments (remember that comments start with the sign #), and finish all python commands. Note that we calculate the multi-year seasonal variability using a weekly calculation.

In [ ]:
# Let's first select the data for analysis. Data will be written in a new dataframe. 
# This command is written for you. 
# You can use '#' sign to exclude the data you do not want to analyse. That way you can slowly build up your analysis
# from the data including only F1 (first row), data including wet canopy (second row), data including 
# variable LAI (thirs row) and data including all stress functions (last row).
df_meteo1_seasonal = df_meteo1[[
    'F_NH3_W', 'F_NH3_V', 'F_NH3_Z',
#    'F_wet_canopy_NH3_W', 'F_wet_canopy_NH3_V', 'F_wet_canopy_NH3_Z', 
#    'F_lai_NH3_W', 'F_lai_NH3_V', 'F_lai_NH3_Z', 
    'F_stress_NH3_W', 'F_stress_NH3_V', 'F_stress_NH3_Z']]

# Calculate the multi-year seasonal variability for all three stations. 
# Multi-year seasonal variability is calculated as a mean of all e.g., first weeks of the year.
df_meteo1_mean_seasonal = 

# Let's make a plot.
# Plot the result (inter-annual variability) for the three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [years] on x-axis and Deposition rate [um/m^2/s] on y-axis.  
fig7 = df_meteo1_mean_seasonal.iplot(asFigure=True, xTitle="Time [weeks]", yTitle="Deposition rate [um/m3/s]", 
    colors=['orange', 'blue', 'green', 'orange', 'blue', 'green'], 
    dash=['solid', 'solid', 'solid', 'dot', 'dot', 'dot'], width=2)
fig7.show()

### Question 4.4b: How does the deposition vary on a seasonal scale when we transition from a simple to a realistic canopy resistance calculation?

#### Your answer is:

### Exercise 4.4c: Diurnal variability

In this exercise, you will look at the variability in the deposition rate on seasonal scale. Do you think you are able to set the python code for yourself? Try, if you encounter a problem, you can always ask for help! 

### Question 4.4c: How does the deposition vary daily when we transition from a simple to a realistic canopy resistance calculation?

#### Your answer is:

## 4.4 Parameter uncertainty and missing processes

Python allows us to calculate the resistance and deposition rates with many decimals, suggesting high accuracy. However, the real accuracy is far less determined by the numerical accuracy than by the accuracy of the (meteorological and air quality) input data, the choice of parameter values and the processes included in the model description. In the discussion section of your PGO report, you will have to consider sources of uncertainty. To prevent you from writing vague sentences like ‘the deposition fluxes are uncertain, because we did not know the exact parameter values’, do some sensitivity studies, where you vary the value of parameters with -10% and +10% or where you exclude/include some processes (e.g., deposition on wet canopy). Note how much the deposition flux changes. If this is less than 10%, the model may not be very sensitive to that parameter or process. But there will also be parameters or processes the model is more than proportionally sensitive to.

In [ ]:
### DO NOT TOUCH!!! WORK IN PROGRESS!!!
oink = df_meteo.copy()

@interact(a=(0., 3.))
def plot_T(a):
    oink['F_NH3_V_pert'] = a * oink['F_NH3_V']
    fig_oink = oink[['F_NH3_V','F_NH3_V_pert']].iplot(asFigure=True)
    fig_oink.show()